In [ ]:
from category_encoders import BinaryEncoder

# Group by gene and kind, and then aggregate
upgenevsrep["position"] = upgenevsrep["position"].astype(float)
# Group by gene and kind, and then aggregate
grouped_df = upgenevsrep.groupby(['gene', 'kind']).agg(
    mean_position=('position', 'mean'),
    var_position=('position', 'var'),
    counts=('counts', 'sum'),
).reset_index()

grouped_df["var_position"] = grouped_df["var_position"].fillna(0.0).round(2)
grouped_df["mean_position"] = grouped_df["mean_position"].round(2)

In [ ]:
quantile_transformer = QuantileTransformer(output_distribution='normal', n_quantiles=400, random_state=0)
oe_transformer = OrdinalEncoder()
be_transformer = BinaryEncoder()

mean_quantile_transf = quantile_transformer.fit_transform(grouped_df["mean_position"].values.reshape(-1, 1))
var_quantile_transf = quantile_transformer.fit_transform(grouped_df["var_position"].values.reshape(-1, 1))
count_quantile_transf = quantile_transformer.fit_transform(grouped_df["counts"].values.reshape(-1, 1))

gene_transf = be_transformer.fit_transform(grouped_df["gene"].values.reshape(-1, 1))
gene_transf.columns = ["gene_1", "gene_2", "gene_3", "gene_4", "gene_5", "gene_6", "gene_7", "gene_8"]
kind_transf = be_transformer.fit_transform(grouped_df["kind"].values.reshape(-1, 1))
kind_transf.columns = ["kind_1", "kind_2", "kind_3"]

grouped_df["transf_mean"] = mean_quantile_transf
grouped_df["transf_var"] = var_quantile_transf
grouped_df["transf_count"] = count_quantile_transf
grouped_df = pd.concat([grouped_df, 
                        gene_transf, 
                        kind_transf], axis=1)

sns.scatterplot(data=grouped_df, x="transf_mean", y="transf_var", hue="kind")
plt.title("Mean and Variance Position")
plt.xlabel("Mean Position")
plt.ylabel("Variance Position")
plt.show();

sns.scatterplot(data=grouped_df, x="transf_mean", y="transf_count", hue="kind")
plt.title("Mean Position vs Counts")
plt.xlabel("Mean Position")
plt.ylabel("Counts")
plt.show();

In [ ]:
from sklearn.metrics import silhouette_score
import joblib

umap_df = grouped_df[[ 'transf_var', 'transf_count', 'transf_mean', 
                      'gene_1', 'gene_2', 'gene_3', 'gene_4', 'gene_5', 'gene_6', 'gene_7', 'gene_8', 
                      'kind_1', 'kind_2', 'kind_3']].copy()

for i in [25, 26, 27]:

    model = umap.UMAP(n_neighbors=i,
                     min_dist=0.0,
                     n_components=3).fit(umap_df)

    clusterable_embedding = model.transform(umap_df)    

    # save the model to disk
    filename = f'./models/trained_models/upgene_umap_n_neighbor_{i}.sav'
    joblib.dump(model, filename)

    labels = HDBSCAN(min_samples=10,
                     min_cluster_size=20,
                     ).fit_predict(clusterable_embedding)
    
    ncluster = len(np.unique(labels))
    score = round(silhouette_score(umap_df, labels),6)
    # save the model to disk
    filename = f'./models/trained_models/upgene_hdbscan_umap_{i}.sav'
    joblib.dump(model, filename)
    
    fig = plt.figure(figsize=(10, 6), dpi=300)

    # Plot UMAP in 3D
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], clusterable_embedding[:, 2], s=5)
    ax2.set_xlabel('')
    ax2.set_ylabel('')
    ax2.set_zlabel('')
    # Plot UMAP in 3D
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], clusterable_embedding[:, 2], s=5, c=labels)
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    ax1.set_zlabel('')

    plt.suptitle(f'UMAP n_neighbors={i}, ncluster={ncluster}, sil. score {score}')
    plt.savefig(f"./results/figures/upgenvsrep_umap_{i}.png")

In [ ]:
 # load the model from disk
filename = f'./models/trained_models/upgene_umap_n_neighbor_26.sav'
loaded_model = joblib.load(filename)
result = loaded_model.transform(umap_df)

labels = HDBSCAN(min_samples=10,
                     min_cluster_size=20,
                     ).fit_predict(result)
    
ncluster = len(np.unique(labels))
score = round(silhouette_score(result, labels),6)

grouped_df["labels"] = labels
percentages = grouped_df.groupby(['labels', 'gene']).agg(
    counts=('gene', 'size'),
    agg_kind=('kind', lambda x: ', '.join(sorted(x.unique())))
).reset_index()
percentages["perc"] = round((percentages["counts"]/sum(percentages["counts"]))*100, 2)
percentages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import umap
from scipy.stats import sem
from tqdm import tqdm

# Parameters
n_neighbors_list = [5, 10, 12, 13, 14, 15, 16, 20, 22, 25, 26]
n_trials = 50

# Initialize dictionary to store results
results = {n: [] for n in n_neighbors_list}

# Main loop with progress bar
for i in tqdm(n_neighbors_list, desc='n_neighbors'):
    for _ in range(n_trials), desc=f'Trials for n_neighbors={i}', leave=False):
        # Perform UMAP
        clusterable_embedding = umap.UMAP(
                                n_neighbors=i,
                                min_dist=0.0,
                                n_components=3).fit_transform(umap_df)

        # Perform HDBSCAN
        labels = HDBSCAN(min_samples=10,
                                 min_cluster_size=20,
                                 ).fit_predict(clusterable_embedding)

        # Calculate the number of clusters
        ncluster = len(np.unique(labels))

        # Calculate silhouette score
        if ncluster > 1:
            score = round(silhouette_score(umap_df, labels), 3)
        else:
            score = -1  # Assign a default bad score if there's only one cluster

        # Store the score
        results[i].append(score)

# Calculate statistics
stats = {
    'n_neighbors': [],
    'mean_score': [],
    'variance_score': [],
    'sem_score': []  # Standard Error of the Mean
}

for n in n_neighbors_list:
    scores = results[n]
    stats['n_neighbors'].append(n)
    stats['mean_score'].append(np.mean(scores))
    stats['variance_score'].append(np.var(scores))
    stats['sem_score'].append(sem(scores))

# Convert to DataFrame
stats_df = pd.DataFrame(stats)

# Plotting
plt.figure(figsize=(10, 6))
plt.errorbar(stats_df['n_neighbors'], stats_df['mean_score'], yerr=stats_df['sem_score'], fmt='-o')
plt.xlabel('n_neighbors')
plt.ylabel('Mean Silhouette Score')
plt.title('Mean Silhouette Score vs. n_neighbors with Confidence Interval')
plt.grid(True)
plt.show()

# Print DataFrame
print(stats_df)